In [1]:
using Random
using Dates
using Optimization
using Lux
using DiffEqFlux: NeuralODE, ADAMW, swish
using DifferentialEquations
using ComponentArrays
using BSON: @save, @load

include(joinpath("..", "src", "delhi.jl"))
include(joinpath("..", "src", "figures.jl"))


plot_extrapolation (generic function with 1 method)

In [2]:


# Define a custom residual block
mutable struct ResidualBlock
    inner_layer::Chain
end

function ResidualBlock(input_dim, hidden_dim)
    return ResidualBlock(Chain(
        Dense(input_dim, hidden_dim, swish),
        Dense(hidden_dim, input_dim)
    ))
end

# Define your neural network with residual blocks
#data_dim = 10  # Adjust this to your input data dimension
#swish(x) = x * σ.(x)

# To forward pass through the network, just call model(x) where x is your input data.


ResidualBlock

In [3]:
function neural_ode(t, data_dim)
    f = Lux.Chain(
            Lux.Dense(data_dim, 256, swish),
            Lux.Dense(256, 128, swish),
            Lux.Dense(128,64,swish),
            Lux.Dense(64, 32, swish),
            Lux.Dense(32, data_dim)
        )

    node = NeuralODE(
        f, extrema(t), Tsit5(),
        saveat=t,
        abstol=1e-9, reltol=1e-9
    )
    
    rng = Random.default_rng()
    p, state = Lux.setup(rng, f)

    return node, ComponentArray(p), state
end

neural_ode (generic function with 1 method)

In [4]:
function train_one_round(node, θ, state, y, opt, maxiters, rng, y0=y[:, 1]; kwargs...)
    predict(θ) = Array(node(y0, θ, state)[1])
    loss(θ) = sum(abs2, predict(θ) .- y)
    
    adtype = Optimization.AutoZygote()
    optf = OptimizationFunction((θ, p) -> loss(θ), adtype)
    optprob = OptimizationProblem(optf, θ)
    res = solve(optprob, opt, maxiters=maxiters; kwargs...)
    res.minimizer, state
end


train_one_round (generic function with 2 methods)

In [5]:
function train(t, y, obs_grid, maxiters, lr, rng, θ=nothing, state=nothing; kwargs...)
    log_results(θs, losses) =
        (θ, loss) -> begin
        push!(θs, copy(θ))
        push!(losses, loss)
        false
    end

    θs, losses = ComponentArray[], Float32[]
    for k in obs_grid
        node, θ_new, state_new = neural_ode(t, size(y, 1))
        if θ === nothing θ = θ_new end
        if state === nothing state = state_new end

        θ, state = train_one_round(
            node, θ, state, y, ADAMW(lr), maxiters, rng;
            callback=log_results(θs, losses),
            kwargs...
        )
    end
    final_loss=0
    θs, state, losses, final_loss
 
end



train (generic function with 3 methods)

In [6]:
@info "Fitting model..."
rng = MersenneTwister(123)
df = Delhi.load()
plt_features = Delhi.plot_features(df)
savefig(plt_features, joinpath("plots", "features.svg"))

df_2016 = filter(x -> x.date < Date(2016, 1, 1), df)
plt_2016 = plot(
    df_2016.date,
    df_2016.meanpressure,
    title = "Mean pressure, before 2016",
    ylabel = Delhi.units[4],
    xlabel = "Time",
    color = 4,
    size = (600, 300),
    label = nothing,
    right_margin=5Plots.mm
)
savefig(plt_2016, joinpath("plots", "zoomed_pressure.svg"))


┌ Info: Fitting model...
└ @ Main c:\Users\SpOoKyJaRvIs\Desktop\CSO211 project\neural-ode-weather-forecast\scripts\heavy2 - Copy.ipynb:1


"c:\\Users\\SpOoKyJaRvIs\\Desktop\\CSO211 project\\neural-ode-weather-forecast\\scripts\\plots\\zoomed_pressure.svg"

In [7]:
t_train, y_train, t_test, y_test, (t_mean, t_scale), (y_mean, y_scale) = Delhi.preprocess(df)

([-1.60579308398432, -1.4367622330383107, -1.267731382092763, -1.0987005311472149, -0.9296696802012057, -0.7606388292556577, -0.5916079783101097, -0.4225771273641006, -0.2535462764185526, -0.08451542547300459, 0.08451542547300459, 0.2535462764185526, 0.4225771273641006, 0.5916079783101097, 0.7606388292556577, 0.9296696802012057, 1.0987005311472149, 1.267731382092763, 1.4367622330383107, 1.60579308398432], [-1.763925125632563 -1.1235556296843154 … 0.9028695693184288 0.8197374527372693; 0.7428026273433367 0.6737312549046214 … 0.09770117510549325 0.007224485672476749; -1.2549872329391258 0.1484330904176096 … 0.17740293507960775 0.2501049921257615; 1.3575622301481491 1.1122878498059419 … -1.1776132360481055 -0.8698951253162611], [1.774823934929868, 1.943854785875416, 2.1128856368214253, 2.2819164877669733, 2.450947338712521, 2.6199781896585304, 2.7890090406040784, 2.9580398915496264, 3.1270707424956354, 3.2961015934411835  …  5.493502655735152, 5.662533506681161, 5.831564357626709, 6.00059

In [8]:
plt_split = plot(
    reshape(t_train, :), y_train',
    linewidth = 3, colors = 1:4,
    xlabel = "Normalized time", ylabel = "Normalized values",
    label = nothing, title = "Pre-processed data"
)
plot!(
    plt_split, reshape(t_test, :), y_test',
    linewidth = 3, linestyle = :dash,
    color = [1 2 3 4], label = nothing
)

plot!(
    plt_split, [0], [0], linewidth = 0,
    label = "Train", color = 1
)
plot!(
    plt_split, [0], [0], linewidth = 0,
    linestyle = :dash, label = "Test",
    color = 1
)
savefig(plt_split, joinpath("plots", "train_test_split.svg"))


"c:\\Users\\SpOoKyJaRvIs\\Desktop\\CSO211 project\\neural-ode-weather-forecast\\scripts\\plots\\train_test_split.svg"

In [9]:
obs_grid = 4:4:length(t_train) # we train on an increasing amount of the first k obs
maxiters = 200
lr = 5e-5
θs, state, losses, final_loss = train(t_train, y_train, obs_grid, maxiters, lr, rng, progress=true);
@save "artefacts/training_output.bson" θs losses

predict(y0, t, θ, state) = begin
    node, _, _ = neural_ode(t, length(y0))
    ŷ = Array(node(y0, θ, state)[1])
end


loss: 209:   2%|█                                       |  ETA: 0:03:36

loss: 208:   2%|█                                       |  ETA: 0:04:12

loss: 207:   2%|██                                      |  ETA: 0:04:31

loss: 206:   3%|██                                      |  ETA: 0:04:40

loss: 205:   4%|██                                      |  ETA: 0:04:49

...

loss: 64.6:  98%|███████████████████████████████████████|  ETA: 0:00:07

loss: 64.6:  99%|███████████████████████████████████████|  ETA: 0:00:05

loss: 64.6: 100%|███████████████████████████████████████|  ETA: 0:00:02

loss: 64.6: 100%|███████████████████████████████████████| Time: 0:07:40


predict (generic function with 1 method)

In [10]:
function plot_pred(
    t_train, y_train, t_grid,
    rescale_t, rescale_y, num_iters, θ, state, loss, y0=y_train[:, 1]
)
    ŷ = predict(y0, t_grid, θ, state)
    plt = plot_result(
        rescale_t(t_train),
        rescale_y(y_train),
        rescale_t(t_grid),
        rescale_y(ŷ),
        loss,
        num_iters
    )
end

@info "Generating training animation..."
num_iters = length(losses)
t_train_grid = collect(range(extrema(t_train)..., length=500))
rescale_t(x) = t_scale .* x .+ t_mean
rescale_y(x) = y_scale .* x .+ y_mean
plot_frame(t, y, θ, loss) = plot_pred(
    t, y, t_train_grid, rescale_t, rescale_y, num_iters, θ, state, loss
)
anim = animate_training(plot_frame, t_train, y_train, θs, losses, obs_grid);
gif(anim, "plots/training.gif")

@info "Generating extrapolation plot..."
t_grid = collect(range(minimum(t_train), maximum(t_test), length=500))
ŷ = predict(y_train[:,1], t_grid, θs[end], state)
plt_ext = plot_extrapolation(
    rescale_t(t_train),
    rescale_y(y_train),
    rescale_t(t_test),
    rescale_y(y_test),
    rescale_t(t_grid),
    rescale_y(ŷ)
);
savefig(plt_ext, "plots/extrapolation.svg")

@info "Done!"

┌ Info: Generating training animation...
└ @ Main c:\Users\SpOoKyJaRvIs\Desktop\CSO211 project\neural-ode-weather-forecast\scripts\heavy2 - Copy.ipynb:16


┌ Info: Saved animation to c:\Users\SpOoKyJaRvIs\Desktop\CSO211 project\neural-ode-weather-forecast\scripts\plots\training.gif
└ @ Plots C:\Users\SpOoKyJaRvIs\.julia\packages\Plots\QZRtR\src\animation.jl:156
┌ Info: Generating extrapolation plot...
└ @ Main c:\Users\SpOoKyJaRvIs\Desktop\CSO211 project\neural-ode-weather-forecast\scripts\heavy2 - Copy.ipynb:27


┌ Info: Done!
└ @ Main c:\Users\SpOoKyJaRvIs\Desktop\CSO211 project\neural-ode-weather-forecast\scripts\heavy2 - Copy.ipynb:40


In [11]:
ŷ = predict(y_train[:,1], t_test, θs[end], state)

4×32 Matrix{Float64}:
 -1.76393   -1.01728   -0.332506   …   0.170147    0.165537    0.16076
  0.742803   0.184449  -0.0922492      0.0523399   0.0538947   0.0552091
 -1.25499   -0.061376   0.270782      -0.101343   -0.104119   -0.106602
  1.35756    0.808868   0.279136      -0.303868   -0.31226    -0.320654

In [12]:
function mean_squared_error(ŷ, y_test)
    diff = ŷ - y_test
    mse = sum(diff .^ 2) / length(diff)
    return mse
end

mean_squared_error (generic function with 1 method)

In [13]:
mean_squared_error(ŷ, y_test)

8.07682245402914